# Approach
Spelling correction can be handled word by word according to the potential mistakes (edit distance).

However, to handle the context, we need some sort of Language Modeling.
Since the OCR mistakes are unpredictable, we cannot depend on known vocabulary at the input. If we do so we suffer a lot of Out-of-Vocabulary (OOV). So we have to do it at the character level.

The problem falls under unmatched sequence learning. Seq2seq models are the choice to solve such problems, as used in machine translation.

Using char level seq2seq yields good results, but has two issues:

1- Not good with long sequences > 50 char
2- Sometimes it produces "Halucination", replacing a word completely with other valid words according to the language model, instead of correcting the word.

Example: Date of acdent--> Seq2Seq --> Date of birth.

The purpose of this notebook is to correct the words in the classical way (based on edit distance), but taking the language model into account.

First we build n-gram LM, which yields a transition probability matrix (A). Then build a spelling corrector based on edit distance, which gives emission probability matrix (B). Finally, we use a Viterbi decoder to find the best sequence of corrections for a given sentence.

# Build word spelling corrector (autocorrect)

In [1]:
from __future__ import print_function
import tensorflow as tf
import keras.backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate, GRU, Dot, TimeDistributed, Activation, Embedding
from keras import optimizers
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import json
from nltk.tokenize import word_tokenize
import re
import os
import tarfile
import csv
import pandas as pd
%matplotlib inline

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_data_with_gt(file_name, delimiter='\t', gt_index=1, prediction_index=0):
    input_texts = []
    gt_texts = []
    for row in open(file_name, encoding='utf8'):
        sents = row.split(delimiter)
        if (len(sents) < 2):
            continue
        input_text = sents[prediction_index]
        gt_texts.append(sents[gt_index])
    return input_texts, gt_texts

In [3]:
def load_raw_data(file_name):
    with open(file_name, 'r') as f:
        return(f.read())

In [4]:
def load_medical_terms(json_file):
    texts = []
    with open(json_file) as f:
        med_terms_dict = json.load(f)
    texts += list(med_terms_dict.keys())
    texts += list(med_terms_dict.values())
    return texts

In [5]:
def load_accidents_terms(file_name):

    f = open(file_name, encoding='utf8')
    line = 0  
    texts = []
    try:
        for r in f:
            for term in r.split('|'):
                    texts += term.replace('\"', '')
    except:
        print('finished')

                
    return texts

In [6]:
def process_word(word):
    # Try to correct the word from known dict
    #word = spell(word)
    # Option 1: Replace special chars and digits
    #processed_word = re.sub(r'[\\\/\-\—\:\[\]\,\.\"\;\%\~\(\)\{\}\$\#\?\●\@\+\-\*\d]', r'', w.lower())
    
    # Option 2: skip all words with special chars or digits
    if(len(re.findall(r'[\\\/\-\—\:\[\]\,\.\"\;\%\~\(\)\{\}\$\#\?\●\@\+\-\*\d]', word.lower())) == 0):
        processed_word = word
    else:
        processed_word = 'UNK'

    # Skip stop words
    #stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]        
    stop_words = []        
    if processed_word in stop_words:
        processed_word = 'UNK'
        
    return processed_word

## Load data

In [7]:
data_path = '../../dat/'
texts = []

In [8]:
# Load tesseract correction

files_list = ['all_ocr_data_2.txt', 
              'field_class_21.txt', 
              'field_class_22.txt', 
              'field_class_23.txt', 
              'field_class_24.txt', 
              'field_class_25.txt', 
              'field_class_26.txt', 
              'field_class_27.txt', 
              'field_class_28.txt', 
              'field_class_29.txt', 
              'field_class_30.txt', 
              'field_class_31.txt', 
              'field_class_32.txt', 
              'field_class_33.txt', 
              'field_class_34.txt', 
              'NL-14622714.txt', 
              'NL-14627449.txt', 
              'NL-14628986.txt', 
              'NL-14631911.txt', 
              'NL-14640007.txt']

#files_list = ['all_ocr_data_2.txt']
#files_list = ['field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt', 'field_class_21.txt']

for file_name in files_list:
    tess_correction_data = os.path.join(data_path, file_name)
    _, gt = load_data_with_gt(tess_correction_data)
    texts += gt
    
# Load HW terms

hw_correction_data = os.path.join(data_path, 'handwritten_output.txt')
_, gt = load_data_with_gt(hw_correction_data, delimiter='|', gt_index=0, prediction_index=1)
texts += gt

# Load clean claims forms

num_samples = 10000
file_name = os.path.join(data_path, 'claims.txt')
#texts += load_raw_data(file_name)

# Load Medical Terms dictionary
json_file = os.path.join(data_path, 'abbrevs.json')
texts += load_medical_terms(json_file)

# Load Medical Instruction dictionary
file_name = os.path.join(data_path, 'medical_instructions.txt')
texts += load_raw_data(file_name)

# Load accident terms
file_name = os.path.join(data_path, 'AccidentsL.txt')
texts += load_accidents_terms(file_name)

# Load procedures and tests
file_name = os.path.join(data_path, 'procedures_tests.txt')
texts += load_raw_data(file_name)

# Load Diagnosis Descriptions
file_name = os.path.join(data_path, 'ICD10.csv')
t = pd.read_csv(file_name)
texts += list(t['DESCRIPTION'].values)

finished


In [9]:
# Sample data
print(len(texts))
for i in range(10):
    print(texts[i], '\n')

1163183
Claim Type: VB Accident - Accidental Injury
 

Who The Reported Event Happened To: Employee/Policyholder
 

Policyholder/Owner Information
 

First Name:
 

Middle Name/Initial:
 

Last Name:
 

Social Security Number:
 

Birth Date:
 

Gender:
 

Language Preference:
 



In [10]:
with open('med.txt', 'w') as f:
    for text in texts:
        f.write(text)
f.close()

In [11]:
!tar -xvf autocorrect/words.tar

words/
words/en_US_GB_CA_mixed.txt
words/big_orig.txt
words/._big.txt
words/big.txt
words/en_US_GB_CA_lower.txt


In [12]:
#f_big_orig = open('words/big_orig.txt', 'r')
f_big_orig = open('med.txt', 'r')
f_med = open('med.txt', 'r')
f_big = open('words/big.txt', 'w')
for line in f_big_orig:
    f_big.write(line + '\n')
for line in f_med:
    f_big.write(line + '\n')
    

f_big_orig.close()
f_big.close()
f_med.close()

In [13]:
!tar -cvf autocorrect/words.tar words

words/
words/en_US_GB_CA_mixed.txt
words/big_orig.txt
words/._big.txt
words/big.txt
words/en_US_GB_CA_lower.txt


In [14]:
!rm -rf words/

# Build n-gram LM

In [15]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from nltk import word_tokenize
from collections import Counter, defaultdict
import random

In [16]:
def lower_case(w):
    if w is not None: w = w.lower() 
    return w

In [17]:
def build_trigram_lm(corpus):
    model = defaultdict(lambda: defaultdict(lambda: 0))

    for sentence in corpus:
        sentence = word_tokenize(sentence)
        #print(sentence)
        #print(word_tokenize(sentence))
        #print(list(trigrams(sentence, pad_right=True, pad_left=True)))
        #print(list(trigrams(word_tokenize(sentence), pad_right=True, pad_left=True)))
        for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
            model[(w1, w2)][w3] += 1

    '''
    print(model["what", "the"]["economists"])  # "economists" follows "what the" 2 times
    print(model["what", "the"]["nonexistingword"])  # 0 times
    print(model[None, None]["The"])  # 8839 sentences start with "The"
    '''
    # Let's transform the counts to probabilities
    for w1_w2 in model:
        total_count = float(sum(model[w1_w2].values()))
        for w3 in model[w1_w2]:
            model[w1_w2][w3] /= total_count
    return model


In [18]:
def build_bigram_lm(corpus):
    model = defaultdict(lambda: defaultdict(lambda: 0))

    for sentence in corpus:
        sentence = word_tokenize(sentence)
        #print(sentence)
        #print(word_tokenize(sentence))
        #print(list(trigrams(sentence, pad_right=True, pad_left=True)))
        #print(list(trigrams(word_tokenize(sentence), pad_right=True, pad_left=True)))
        for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
            w1 = lower_case(w1)
            w2 = lower_case(w2)
            model[w1][w2] += 1

    # Let's transform the counts to probabilities
    for w1 in model:
        w1 = lower_case(w1)
        total_count = float(sum(model[w1].values()))
        for w2 in model[w1]:
            w2 = lower_case(w2)
            model[w1][w2] /= total_count
    return model


In [19]:

p = build_bigram_lm(texts)

In [20]:
p['first']['name']

0.11124845488257108

In [21]:
w = 'First'
w = None
if w is not None: w = w.lower() 
w

# Viterbi decoder

In [22]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = V[t-1][states[0]]["prob"]*trans_p[states[0]][st]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t-1][prev_st]["prob"]*trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
                    
            max_prob = max_tr_prob * emit_p[st][obs[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}
    '''
    for line in dptable(V):
        print(line)
    '''    
    opt = []
    # The highest probability
    max_prob = max(value["prob"] for value in V[-1].values())
    previous = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st)
            previous = st
            break
    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]

    #print('The steps of states are ' + ' '.join(opt) + ' with highest probability of %s' % max_prob)

    return opt

def dptable(V):
    # Print a table of steps from dictionary
    yield(" ".join(("%12d" % i) for i in range(len(V))))
    for state in V[0]:
        yield("%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V))

# Sentence path decoder
Given a sequence of words:

- Build states: set of all possible states at each word location. 
s_K, K=number of states=All possible candidates at all words locations.
s is a dict, keys=words, values=state index

- Get the emission probability for each observed word from the states set. 
B_TxK, T=number of words in the sentence.
B is a an array of dicts.
B[i] is dict: key=word from s_k, value=emission probability score for each word from s_K, if the input is w_i (NLP_COUNT.get(candidate))

- Build state transition matrix A_KxK.
use p, the n-gram LM.
double loop on all states (word from s_k), and get for each A[si][sj]=p[si][sj]


- Build initial states array Pi_K.
Pi_K is a dict, keys=word from s_k, value=scores (normalized) of the candidates of word at loc i=0--> B[0]


- Run Viterbi on:
```
obs = nltk.tokenize(sentence)
states = s_K
trans_p=A
emit_p=B
output = viterbi(obs, states, start_p, trans_p, emit_p)
```

In [23]:
from autocorrect import spell
from autocorrect.nlp_parser import NLP_COUNTS
from autocorrect.word import Word, common, exact, known, get_case

In [24]:
import numpy as np
from nltk.tokenize import word_tokenize

In [25]:
def get_candidates(word):
    w = Word(word)
    
    candidates = (common([word]) | exact([word]) | known([word]) |
              known(w.typos()) | common(w.double_typos()) or
              [word])

    
    
    '''
    candidates = (common([word]) | exact([word]) | known([word]) |
              known(w.typos()) or common(w.double_typos()) or
              [word])
    '''
    '''
    candidates = (common([word]) or exact([word]) or known([word]) or
                  known(w.typos()) or common(w.double_typos()) or
                  [word])
    '''
    candidates_dict = {}
    for candidate in candidates:
        candidates_dict[candidate] = NLP_COUNTS.get(candidate)
    return candidates_dict

In [26]:
def get_candidates(word):
    w = Word(word)
    
    candidates = (common([word]) | exact([word]) | known([word]) |
              known(w.typos()) | common(w.double_typos()) or
              [word])

    w_common = 10
    w_exact = 10
    w_known = 5
    w_known_typo = 2
    w_known_double_typos = 0.05
    w_word = 0.5
    
    tot = w_common + w_exact + w_known + w_known_typo + w_known_double_typos + w_word
    w_common /= tot
    w_exact  /= tot
    w_known  /= tot
    w_known_typo  /= tot
    w_known_double_typos  /= tot
    w_word  /= tot
    
    candidates_dict = {}
    for i, candidate in enumerate(common([word])):
        candidates_dict[candidate] = w_common * NLP_COUNTS.get(candidate)
    for i, candidate in enumerate(exact([word])):
        candidates_dict[candidate] = w_exact * NLP_COUNTS.get(candidate)                                  
    for i, candidate in enumerate(known([word])):
        candidates_dict[candidate] = w_known * NLP_COUNTS.get(candidate)                                  
    for i, candidate in enumerate(known(w.typos())):
        candidates_dict[candidate] = w_known_typo * NLP_COUNTS.get(candidate)                                  
    for i, candidate in enumerate(common(w.double_typos())):
        candidates_dict[candidate] = w_known_double_typos * NLP_COUNTS.get(candidate) 
    candidates_dict[word] = w_word * NLP_COUNTS.get(word) 
                                  
    return candidates_dict

In [27]:
word = 'Nane'

candidates_dict = get_candidates(word)

s = set()
s |= set(candidates_dict.keys())
#s
exact([word])

set()

In [28]:
common([word])

set()

In [29]:
exact([word])

set()

In [30]:
known([word])

{'nane'}

In [31]:
w = Word(word)
known(w.typos())

{'ane',
 'anne',
 'bane',
 'cane',
 'dane',
 'fane',
 'gane',
 'inane',
 'jane',
 'kane',
 'lane',
 'mane',
 'nabe',
 'nace',
 'nae',
 'nage',
 'nake',
 'nale',
 'name',
 'nan',
 'nana',
 'nance',
 'nand',
 'nane',
 'nanes',
 'nani',
 'nanp',
 'nans',
 'nant',
 'nape',
 'nare',
 'nate',
 'nave',
 'naze',
 'nene',
 'nine',
 'nne',
 'none',
 'pane',
 'rane',
 'sane',
 'tane',
 'vane',
 'wane',
 'zane'}

In [32]:
common(w.double_typos())

{'age',
 'an',
 'and',
 'ang',
 'ank',
 'ant',
 'any',
 'are',
 'ave',
 'axe',
 'bale',
 'band',
 'bank',
 'bare',
 'base',
 'bene',
 'bone',
 'cafe',
 'cage',
 'came',
 'can',
 'canoe',
 'care',
 'case',
 'cave',
 'cone',
 'crane',
 'date',
 'done',
 'dune',
 'ease',
 'face',
 'fame',
 'fan',
 'fine',
 'gade',
 'game',
 'gang',
 'gate',
 'gave',
 'gene',
 'gone',
 'han',
 'hand',
 'hang',
 'have',
 'igne',
 'lake',
 'lance',
 'land',
 'late',
 'line',
 'lune',
 'made',
 'make',
 'male',
 'man',
 'mane',
 'maneq',
 'many',
 'mine',
 'na',
 'nail',
 'nal',
 'name',
 'named',
 'names',
 'napa',
 'nee',
 'nice',
 'nine',
 'node',
 'non',
 'none',
 'nose',
 'note',
 'one',
 'pace',
 'page',
 'pale',
 'pan',
 'panel',
 'plane',
 'race',
 'ran',
 'range',
 'rate',
 'safe',
 'sake',
 'same',
 'sand',
 'snake',
 'take',
 'tank',
 'tape',
 'tone',
 'une',
 'van',
 'wake',
 'want',
 'ware',
 'wave',
 'zone'}

In [33]:
get_case(word, 'nane')

'Nane'

In [34]:
B = {}
word = 'Cender'
candidates_dict = get_candidates(word)
for candidate in candidates_dict.keys():
    if not candidate in B:
        B[candidate] = {}
    B[candidate][word] = candidates_dict[candidate]

In [35]:
emit_p = {'a':1, 'b':2}
tot = np.sum(list(emit_p.values()))


In [36]:

sentence = 'Patiet last tet is needing this for school at this'
#sentence = 'First Nane'
sentence = word_tokenize(sentence)

# K = n_states
# N = n_obs
# Build s and emit_p (B_KxN)
states = set()
emit_p = {}
for obs in sentence:
    obs = lower_case(obs)
    candidates_dict = get_candidates(obs)
    states |= set(candidates_dict.keys())
    for state in candidates_dict.keys():
        if not state in emit_p:
            emit_p[state] = {}
        emit_p[state][obs] = candidates_dict[state]

# Normalize emit_p
for state in emit_p.keys():
    #for obs in emit_p[state].keys():
    for obs in sentence:     
        obs = lower_case(obs)
        if obs in emit_p[state].keys():
            emit_p[state][obs] /= (np.sum(list(emit_p[state].values()))+1)
        else:
            emit_p[state][obs] = 0

# Build trans_p (A_KxK)
trans_p = {}
for state_i in states:
    
    if not state_i in trans_p:
        trans_p[state_i] = {}
    
    for state_j in states:
        trans_p[state_i][state_j] = p[state_i][state_j]
        
# Build start_p (Pi_Kx1)
start_p = {}
w_0 = lower_case(sentence[0])
candidates_dict = get_candidates(w_0)
for state in states:
       
    #if state in candidates_dict.keys():
    if state == spell(w_0):
        start_p[state] = candidates_dict[state]
    else:
        start_p[state] = 0

# Normalize start_p
for state in states:
    start_p[state] /= np.sum(list(start_p.values()))

obs = [lower_case(obs) for obs in sentence] 

#print('obs: ' + str(obs))
#print('states: ' + str(states))
#print('start_p: ' + str(start_p))
#print('trans_p: ' + str(trans_p))
#print('emit_p: ' + str(emit_p))
correct_words = viterbi(obs, tuple(states), start_p, trans_p, emit_p)

correct_words_ = []
for i, w in enumerate(sentence):
    correct_words_.append(get_case(w, correct_words[i]))
    
print('Correction: ' + ' '.join(correct_words_))

Correction: Patient was not be needing this to school on the


In [37]:
trans_p['patient']['was']*emit_p['was']['last']

0.004418159562346848

In [38]:
get_candidates('last')['last']

17.858439201451908

In [39]:
get_candidates('last')['was']

12.058076225045372

In [40]:
trans_p['patient']['last']*emit_p['last']['last']

0.0009819011862144665

In [41]:
trans_p['patient']['was']

0.01362088535754824

In [42]:
trans_p['patient']['last']

0.0011350737797956867

In [43]:
spell_corrected_sentence = []
for w in sentence:
    spell_corrected_sentence.append(spell(w))
print(' '.join(spell_corrected_sentence))

Patient last tet is needing this for school at this


In [44]:
emit_p['wrist']['first']

KeyError: 'wrist'

In [ ]:
emit_p['first']['first']

In [ ]:
emit_p['name']['nane']

In [ ]:
emit_p['and']['nane']

In [ ]:
trans_p['wrist']['and']

In [ ]:
trans_p['first']['first']

In [ ]:
trans_p['first']['name']

In [ ]:
start_p['first']

In [ ]:
start_p['wrist']

In [ ]:
start_p['nane']

In [ ]:
emit_p['name']

In [ ]:
get_candidates('nane')['name']

In [ ]:
NLP_COUNTS.get('name')

In [45]:
def spell_viterbi(sentence):
    sentence = word_tokenize(sentence)

    # K = n_states
    # N = n_obs
    # Build s and emit_p (B_KxN)
    states = set()
    emit_p = {}
    for obs in sentence:
        obs = lower_case(obs)
        candidates_dict = get_candidates(obs)
        states |= set(candidates_dict.keys())
        for state in candidates_dict.keys():
            if not state in emit_p:
                emit_p[state] = {}
            emit_p[state][obs] = candidates_dict[state]

    # Normalize emit_p
    for state in emit_p.keys():
        #for obs in emit_p[state].keys():
        for obs in sentence:     
            obs = lower_case(obs)
            if obs in emit_p[state].keys():
                emit_p[state][obs] /= (np.sum(list(emit_p[state].values()))+1)
            else:
                emit_p[state][obs] = 0

    # Build trans_p (A_KxK)
    trans_p = {}
    for state_i in states:

        if not state_i in trans_p:
            trans_p[state_i] = {}

        for state_j in states:
            trans_p[state_i][state_j] = p[state_i][state_j]

    # Build start_p (Pi_Kx1)
    start_p = {}
    w_0 = lower_case(sentence[0])
    candidates_dict = get_candidates(w_0)
    for state in states:

        #if state in candidates_dict.keys():
        if state == spell(w_0):
            start_p[state] = candidates_dict[state]
        else:
            start_p[state] = 0

    # Normalize start_p
    for state in states:
        start_p[state] /= np.sum(list(start_p.values()))

    obs = [lower_case(obs) for obs in sentence] 

    #print('obs: ' + str(obs))
    #print('states: ' + str(states))
    #print('start_p: ' + str(start_p))
    #print('trans_p: ' + str(trans_p))
    #print('emit_p: ' + str(emit_p))
    correct_words = viterbi(obs, tuple(states), start_p, trans_p, emit_p)

    correct_words_ = []
    for i, w in enumerate(sentence):
        correct_words_.append(get_case(w, correct_words[i])) 
    return ' '.join(correct_words_)

In [46]:
c = spell_viterbi('Who The Reported Event Happened To: Employee/Policyholder')
c

'Who The Reported Event Happened To my My'

In [47]:
for text in texts:
    print('GT: ' + text)
    print('Correction: ' + spell_viterbi(text))

GT: Claim Type: VB Accident - Accidental Injury

Correction: Claim Type i OR Accident to Accidental Injury
GT: Who The Reported Event Happened To: Employee/Policyholder

Correction: Who The Reported Event Happened To my My
GT: Policyholder/Owner Information



/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars


KeyError: None